In [4]:
!pip install longchain
!pip install longchain_openai
!pip install transformers
!pip install langchain.embeddings
!pip install langchain-community langchain-core
!pip install gradio
!pip install langchain_chroma


ERROR: Could not find a version that satisfies the requirement longchain_openai (from versions: none)
ERROR: No matching distribution found for longchain_openai
ERROR: Could not find a version that satisfies the requirement langchain.embeddings (from versions: none)
ERROR: No matching distribution found for langchain.embeddings
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [6]:
import zipfile
import os

zip_path = "/content/books_db.zip"
extract_path = "/content/books_db"

if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)


In [17]:

import gradio as gr
import pandas as pd
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import re

books = pd.read_csv("/content/cleaned_data.csv")

embed_model = HuggingFaceEmbeddings(model_name='PartAI/Tooka-SBERT-V2-Large')

books_db = Chroma(
    embedding_function=embed_model,
    persist_directory='/content/books_db/content/PERSIST_DIRECTORY'
)


In [8]:
import re
import pandas as pd

def search_books(query, books_db, books, k=10):
    docs = books_db.similarity_search(query, k=k)

    book_rows = []
    for doc in docs:
        text = doc.page_content.strip()
        match = re.match(r"(\d+)", text)
        if match:
            book_id = int(match.group(1))
            row = books[books["book_id"] == book_id]
            if not row.empty:
                book_rows.append(row)

    if book_rows:
        result_df = pd.concat(book_rows, ignore_index=True)
    else:
        result_df = pd.DataFrame()
    return result_df



In [15]:
query = 'عادت'
result_df = search_books(query, books_db, books, k=10)
display(result_df)

,Unnamed: 0,book_id,title,aurthor,description,poster-src,published year,num pages,tagged_description
0,2372,80133,هنر نه گفتن,دیمون زاهاریادس,بسیاری از ما، از جمله خودم، بیش‌تر عمرمان را ب...,https://img.taaghche.com/frontCover/59595.jpg?...,۱۳۹۸,۱۵۹,80133بسیاری از ما، از جمله خودم، بیش‌تر عمرمان...
1,1437,64410,کاش وقتی بیست ساله بودم می دانستم,تینا سیلیگ,اگر تصمیم به ترک کردن گرفتید مطمئن شوید که آن ...,https://img.taaghche.com/frontCover/57502.jpg?...,۱۳۹۸,۱۹۳,64410اگر تصمیم به ترک کردن گرفتید مطمئن شوید ک...
2,2092,38990,سایه,کارین ژیه‌بل,سه ساعت خواب... کم است... خیلی کم، بااین‌حال، ...,https://img.taaghche.com/frontCover/94493.jpg?...,۱۳۹۷,۵۵۲,38990سه ساعت خواب... کم است... خیلی کم، بااین‌...
3,407,34090,زندگی نزیسته‌ات را زندگی کن,رابرت الکس جانسون,در نیمه‌ی دوم عمر، که معمولا پس از سی سالگی آن...,https://img.taaghche.com/audioCover/22011.jpg,NaN,۰,34090در نیمه‌ی دوم عمر، که معمولا پس از سی سال...
4,4829,29287,تو مانده ای برای من,سمیه هرمزی,«پنجشنبه‌ها از بس لای خاطراتم را ورق می‌زنم، و...,https://img.taaghche.com/frontCover/162079.jpg...,۱۴۰۱,۴۵۶,29287«پنجشنبه‌ها از بس لای خاطراتم را ورق می‌ز...


In [20]:
import gradio as gr
import pandas as pd
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import zipfile
import os
import re


def search_books(query, k=5):
    docs = books_db.similarity_search(query, k=k)
    cards_html = ""
    for doc in docs:
        text = doc.page_content.strip()
        match = re.match(r"(\d+)", text)
        if match:
            book_id = int(match.group(1))
            row = books[books["book_id"] == book_id]
            if not row.empty:
                book = row.iloc[0]
                poster = book['poster-src'] if pd.notna(book['poster-src']) else ""
                title = book['title']
                author = book['aurthor']
                desc = book['description']
                year = book['published year']
                num_pages = book['num pages']
                # ساخت کارت HTML
                cards_html += f"""
                <div style="display:inline-block; width:220px; margin:10px; border:1px solid #eee; border-radius:8px; box-shadow:0 2px 8px #eee; vertical-align:top;">
                    <img src="{poster}" alt="cover" style="width:100%;height:280px;object-fit:cover;border-radius:8px 8px 0 0;">
                    <div style="padding:10px;">
                        <b>{title}</b><br>
                        <small>{author} - {year}</small><br>
                        <span style="color:#888;font-size:12px;">صفحات: {num_pages}</span><br>
                        <div style="color:#555;font-size:13px;height:40px;overflow:hidden;">{desc}</div>
                    </div>
                </div>
                """
    if not cards_html:
        cards_html = "<p> .نتیجه‌ای یافت نشد</p>"
    return cards_html

with gr.Blocks() as demo:
    gr.Markdown("## جستجوی کتاب‌ها بر اساس توضیحات یا عنوان")
    with gr.Row():
        query = gr.Textbox(label="بگو میخوای راجب چی کتاب بهت معرفی کنم؟")
    output = gr.HTML()
    btn = gr.Button("جستجو")
    btn.click(fn=search_books, inputs=query, outputs=output)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ea4f7c0fcaad8f3507.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
